In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.5 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 32.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
     |████████████████████████████████| 212 kB 60.5 MB/s 
     |████████████████████████████████| 140 kB 56.4 MB/s 
     |████████████████████████████████| 127 kB 62.1 MB/s 
     |████████████████████████████████| 271 kB 52.1 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |█████████

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import TrainingArguments, Trainer
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_metric
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
language_code = 'ar'
language_name = 'arabic'
base_model = "facebook/wav2vec2-large-xlsr-53"

data_dir = "./drive/My Drive/Senior Project/Resources/data"
bucket_name = "hearsome-sagemaker-datasets"

output_models_dir = f"./workspace/output_models/{language_code}/wav2vec2-large-xlsr-{language_name}-demo"
new_output_models_dir = f"./drive/My Drive/Senior Project/Resources/workspace/output_models/{language_code}/wav2vec2-large-xlsr-{language_name}"

In [ ]:
train = pd.read_pickle(f"{data_dir}/train.pbz2", compression='bz2')
dev = pd.read_pickle(f"{data_dir}/dev.pbz2", compression='bz2')

In [ ]:
train.head()

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(new_output_models_dir)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MGB3Dataset(Dataset):
    def __init__(self, df):
        super(Dataset, self).__init__()
        self.data = df[["input_values", "input_length", "labels"]].to_dict('records')
        
    def __len__(self):
        return len(self.data)
        

    def __getitem__(self, index):
        return self.data[index]

In [ ]:
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
        
    def __init__(self, processor: Wav2Vec2Processor, padding: Union[bool, str]):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install jiwer

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [ ]:
model.freeze_feature_encoder()

In [ ]:
training_args = TrainingArguments(
  output_dir=new_output_models_dir,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
train_dataset = MGB3Dataset(train)
dev_dataset = MGB3Dataset(dev)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train(
    resume_from_checkpoint=True,
)

In [ ]:
import pandas as pd
from datasets import load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from torch.utils.data import Dataset


class MGB3Dataset(Dataset):
    def __init__(self, df):
        super(Dataset, self).__init__()
        self.data = df[["input_values", "input_length", "labels"]].to_dict('records')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


test = pd.read_pickle(f".{data_dir}/test.pbz2", compression='bz2')
test_dataset = MGB3Dataset(test)


wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("Zaid/wav2vec2-large-xlsr-53-arabic-egyptian")
model = Wav2Vec2ForCTC.from_pretrained("Zaid/wav2vec2-large-xlsr-53-arabic-egyptian")
model.to("cuda")

print("WER: {:2f}".format(100 * wer.compute(predictions=test_dataset["pred_strings"],
                                            references=test_dataset["sentence"])))